In [ ]:
# Import required modules
import pandas as pd
import re                                           # regular expressions
import nltk
import nltk.corpus as corpus                        # nltk package for reading files
nltk.download('stopwords')
from nltk.corpus import stopwords as stopwords      # words such as 'the','be','an', that don't significantly impact the sentiment
from nltk.stem import WordNetLemmatizer             # Groups words
from nltk.tokenize import word_tokenize             # Splits text into tokens
import collections

# Will be used later on
import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# import sklearn
# import numpy as np

In [ ]:
# Filter out all unlabelled sentiment
p_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "positive"')
n_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "negative"')

# Debug Print
# display(p_df)
# display(n_df)

# Combine dataframes
frames = [p_df, n_df]
df = pd.concat(frames)

# Sort by date
# https://www.geeksforgeeks.org/how-to-sort-a-pandas-dataframe-by-date/
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by="created_at",ascending=True)
# Debug Print
display(df)


In [ ]:
# Grab the top 25 stocks
ticker_pattern = re.compile(r'(^\$[A-Z]+|^\$ES_F)')

# Break the string into a dictionary
ticker_freq_dict = collections.defaultdict(int)
# ticker_dict = collections.defaultdict(str)

# Count the frequencies of the mentions of each stock
for text in df['text']:
    for word in text.split():
        if ticker_pattern.fullmatch(word) is not None:
            if "$" + word not in ticker_freq_dict:
                ticker_freq_dict["$" + word[1:]] += 1
    


# Only store the top 25 mentioned stocks
ticker_freq_df = pd.DataFrame.from_dict(ticker_freq_dict, orient='index', columns=['freq']).nlargest(25, 'freq')
display(ticker_freq_df.sort_values('freq', ascending=False))


### **`ticker_dict`** intended structure
There will be a key for each of the 25 most frequently mentioned stocks. Each Stock will have a list of all the sentiments about tweets relating to the stock, and the associated timestamp the tweet was made.
```
ticker_dict = {
    "$SPX": [
        [
            "Timestamp",
            "Sentiment" (positive or negative)
        ]
    ],
    ,
    "$GOOGL": [
        [
            "Timestamp",
            "Sentiment" (positive or negative)
        ]
    ]
}
```

In [ ]:
# Create a dictionary using the above structure
ticker_dict = collections.defaultdict(str)

# Iterate over all the tweets
for text in df['text']:
    # Determine which stocks the tweet is associated with, and store with appropriate stock
    for ticker in ticker_freq_df.index:
        if text.find(ticker) > 0:
            # Put the store the sentiment and rating in a tuple to be stored in a list
            row_df = df.query('text == @text')
            timestamp = str(row_df.iat[0,0])
            sentiment = row_df.iat[0,2]
            entry = [timestamp, sentiment]
            # print(entry)

            if ticker not in ticker_dict:
                ticker_dict[ticker] = [entry]
                continue
            else:
                ticker_dict[ticker].append(entry)
                continue

In [ ]:
# Print out the dictionary to json file for debugging purposes
import json

with open('/home/mia/Documents/College/CS4811/TwitterNLP/reference/ticker_dict.json','w') as fout:
    json.dump(ticker_dict, fout, indent=4)

# # Confirm the correct amount of tweets have been written to the file
# for ticker in ticker_freq_df.index:
#     if len(ticker_dict[ticker]) != ticker_freq_dict[ticker]:
#         print("ERROR: " + ticker + " expected: " + str(ticker_freq_dict[ticker]) + ", Actual: " + str(len(ticker_dict[ticker])))

# with open('entry.json', 'w') as fout:
#     json.dump(ticker_dict['$JNJ'], fout, indent=4)

In [ ]:
# Convert to a ticker_dict toDataframe
sentiment_df = pd.DataFrame.from_dict(ticker_dict, orient='index')
display(sentiment_df.head(5))

### **`sentiment_dict`** intended structure
There will be a key for each of the 25 most frequently mentioned stocks. Each Stock have a list of days an the associated percentage of positive tweets.
```
ticker_dict = {
    "$SPX": [
        "Day": weight,
        "Day": weight,
        ...
    ],
}
```

In [ ]:
# Return a weight from 0-1 based on the percentage of positive tweets
stock_weight_dict = dict()
round_time_pattern = " " + ".*"
current_time = ""
total = 0
total_positive = 0

created = []

for stock in sentiment_df.index:
    tweets = sentiment_df.loc[stock]
       
    for tweet in tweets:
        if tweet is not None:
            # print(tweet)
            # round tweet to the day
            # print(re.sub(round_time_pattern,'', tweet[0]))
                        
            current_time = re.sub(round_time_pattern, '', tweet[0])
            current_time = re.sub(r'-','/', current_time)
            
            if current_time not in stock_weight_dict:
                stock_weight_dict[current_time] = []
            
            # If it is a new date, reset the counter and calculate the percentage
            if current_time not in created:
                # If the total is 0, it is the first round
                if total != 0:
                    # print(current_time)
                    # print(total)
                    # print(total_positive)
                    weight = round(total_positive/total, 2)
                    if weight == .5:
                        weight = 0
                    elif weight < .5:
                        weight = -1
                    elif weight > .5:
                        weight = 1
                    stock_weight_dict[current_time].append({stock: weight})
                    total = 0
                    total_positive = 0
                else:
                    total += 1
                    if tweet[1] == "positive":
                        total_positive += 1
                created.append(current_time)
            else:
                total += 1
                if tweet[1] == "positive":
                    total_positive += 1
                
            # print(created)
    # break
    created.clear()

In [ ]:
# Debug
import pprint
pp = pprint.PrettyPrinter(indent=4)           
pp.pprint(stock_weight_dict)

with open('/home/mia/Documents/College/CS4811/TwitterNLP/reference/sentiment.json', 'w') as fout:
    json.dump(stock_weight_dict, fout, indent=4)

In [ ]:
def stock_to_csv(stock):
    # create a dataframe of just the stock and its date
    counter = 0
    single_stock_df = pd.DataFrame(columns=['Date', stock])
    for date in stock_weight_dict:
        for entries in stock_weight_dict[date]:
            if stock in entries:
                single_stock_df.loc[counter] = [date, entries[stock]]
                counter = counter + 1
                break
    stock = re.sub(r'\$','',stock)
    # print(stock)
    single_stock_df.to_csv("/home/mia/Documents/College/CS4811/TwitterNLP/twt_data/"+stock+".csv",index=False)
    return
    
for stock in ticker_freq_df.index:
    stock_to_csv(stock)

for ticker in ticker_freq_df.index:
    ticker_freq_df.rename(index={ticker: re.sub(r"\$","",ticker)}, inplace=True)
ticker_freq_df.to_csv("/home/mia/Documents/College/CS4811/TwitterNLP/twt_data/stocks_list.csv")